In [1]:
import util
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

In [2]:
#constants
PERCENT_OF_DATA = 0.0125
NUM_ENSEMBLE_MODELS = 3

num_classes = 2
model_save_path = "models/deepfake_detector_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
saliency_partitioned_save_path = "PartitionedDataset/"

# Training ResNet-18 Model

In [ ]:
# load data
train_subset = util.load_data.get_subset('Train', data_percentage=PERCENT_OF_DATA)
test_subset = util.load_data.get_subset('Test', data_percentage=PERCENT_OF_DATA)
validation_subset = util.load_data.get_subset('Validation', data_percentage=PERCENT_OF_DATA)

train_loader = util.load_data.load_dataset(train_subset, batch_size=32)
test_loader = util.load_data.load_dataset(test_subset, batch_size=32)
validation_loader = util.load_data.load_dataset(validation_subset, batch_size=32)

In [ ]:
model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=num_classes)

# Example training setup
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
util.train_model.train_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:00<00:00,  3.28s/it]


Training Loss: 0.4030
Validation Loss: 0.3451, Validation Accuracy: 89.02%

Epoch [2/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [02:58<00:00,  3.25s/it]


...

Training Loss: 0.0582
Validation Loss: 0.3681, Validation Accuracy: 88.62%

Epoch [9/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:04<00:00,  3.36s/it]


Training Loss: 0.0400
Validation Loss: 0.6391, Validation Accuracy: 84.76%

Epoch [10/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [03:06<00:00,  3.39s/it]


Training Loss: 0.0622
Validation Loss: 0.3141, Validation Accuracy: 87.80%



In [11]:
test_loss, test_accuracy = util.train_model.test(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%\n")

Test Loss: 0.2628, Test Accuracy: 84.56%



In [12]:
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to models/deepfake_detector_model.pth


# Disjoint Ensembles Against Adversarial Examples

## Creating Images/Subsets

In [7]:
# create folder for subset of data
util.subset_data.create_subset_from_existing_dataset("Deepfake_Dataset/", "Subset_Dataset/", PERCENT_OF_DATA/4)

Subset dataset created at Subset_Dataset/


In [9]:
def partition_and_save_images(dataset_path, save_path):
    for class_name in ['Real', 'Fake']:
        class_folder = os.path.join(dataset_path, class_name)
        img_list = os.listdir(class_folder)

        print(f"Processing class: {class_name}")
        for img_name in tqdm(img_list, desc=f"Partitioning {class_name}", unit="image"):
            img_path = os.path.join(class_folder, img_name)
            image = util.saliency_partitioning.load_image(img_path)
            partitions = util.saliency_partitioning.partition_image_for_ensemble(single_model, image)
            
            for partition_idx, partition in enumerate(partitions):
                partition = np.clip(partition, 0, 255).astype(np.uint8)
                partition_folder = os.path.join(save_path, f"Partition{partition_idx + 1}", class_name)
                os.makedirs(partition_folder, exist_ok=True)
                partition_image_path = os.path.join(partition_folder, img_name)
                im = Image.fromarray(partition, 'RGB')
                im.save(partition_image_path)

# Example: Partition and save images
partition_and_save_images('Subset_Dataset/Train', save_path='PartitionedDataset/Train')
partition_and_save_images('Subset_Dataset/Validation', save_path='PartitionedDataset/Validation')
partition_and_save_images('Subset_Dataset/Test', save_path='PartitionedDataset/Test')

Processing class: Real


Partitioning Real: 100%|█████████████████████████████████████████████████████████████████████████████████| 218/218 [24:21<00:00,  6.70s/image]


Processing class: Fake


Partitioning Fake: 100%|█████████████████████████████████████████████████████████████████████████████████| 218/218 [24:32<00:00,  6.75s/image]


Processing class: Real


...

Partitioning Fake: 100%|███████████████████████████████████████████████████████████████████████████████████| 61/61 [06:52<00:00,  6.76s/image]


Processing class: Real


Partitioning Real: 100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [01:49<00:00,  6.86s/image]


Processing class: Fake


Partitioning Fake: 100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [02:03<00:00,  7.27s/image]


## Ensemble Model

In [3]:
ensemble_models = []
for i in range(NUM_ENSEMBLE_MODELS):
    model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=2)
    model.load_state_dict(torch.load(model_save_path))
    model.to(device)
    model.eval()  # Set to eval mode
    ensemble_models.append(model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Single Model

In [4]:
single_model = util.dfnn_model.DeepfakeDetectorCNN(num_classes=2)
single_model.load_state_dict(torch.load(model_save_path))
single_model.to(device)
single_model.eval()

DeepfakeDetectorCNN(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

## Comparing
Takes too long.

In [5]:
def create_partitioned_dataloaders(path, subset):
    subset_path = os.path.join(path, subset)
    dataloaders = []
    for part_idx in range(NUM_ENSEMBLE_MODELS):
        part_dataset = util.load_data.get_subset(subset=f"Partition{part_idx + 1}", data_dir=subset_path, data_percentage=1)
        part_dataloader = util.load_data.load_dataset(part_dataset, shuffle=False)
        dataloaders.append(part_dataloader)
    return dataloaders
        
train_dataloaders = create_partitioned_dataloaders(saliency_partitioned_save_path, 'Train')
validation_dataloaders = create_partitioned_dataloaders(saliency_partitioned_save_path, 'Validation')
test_dataloaders = create_partitioned_dataloaders(saliency_partitioned_save_path, 'Test')

In [8]:
for model, train_loader, validation_loader in zip(ensemble_models, train_dataloaders, validation_dataloaders):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    util.train_model.train_with_validation(model, train_loader, validation_loader, criterion, optimizer, device)

Epoch [1/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:31<00:00,  2.24s/it]


Training Loss: 3.2177
Validation Loss: 1.0544, Validation Accuracy: 50.00%

Epoch [2/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:29<00:00,  2.14s/it]


Training Loss: 0.6164
Validation Loss: 2.4579, Validation Accuracy: 49.18%

Epoch [3/10] - Training:


...

Training Loss: 0.6955
Validation Loss: 0.6935, Validation Accuracy: 45.90%

Epoch [7/10] - Training:


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:32<00:00,  2.29s/it]


Training Loss: 0.7033
Validation Loss: 0.6900, Validation Accuracy: 56.56%

Epoch [8/10] - Training:


 14%|███████████████▏                                                                                          | 2/14 [00:05<00:30,  2.56s/it]


KeyboardInterrupt: 